In [3]:
#Framework for performing the experiments for the metro data set

In [1]:
import pandas as pd
from sklearn import linear_model, tree, neural_network, preprocessing
import numpy as np
import random
import math
from datetime import datetime
from matplotlib import pyplot as plt
import copy
import pydot
import graphviz

In [2]:
ord_enc = preprocessing.OrdinalEncoder()
#Import the data set and remove the rows with missing values
metro_data = pd.read_csv('datasets/Metro_Interstate_Traffic_Volume.csv')

#transform the categorical holiday variable into a encoded variable.
metro_data["holiday"] = ord_enc.fit_transform(metro_data[["holiday"]])
# print(metro_data["holiday"].value_counts())


metro_data["weather_main"] = ord_enc.fit_transform(metro_data[["weather_main"]])
# print(metro_data["weather_main"].value_counts())

metro_data["weather_description"] = ord_enc.fit_transform(metro_data[["weather_description"]])
# print(metro_data["weather_description"].value_counts())

# metro_data["date_time"] = ord_enc.fit_transform()
# print(metro_data["weather_description"].value_counts())
metro_data['time'] = pd.to_datetime(metro_data['date_time']).dt.time
metro_data['time'] = ord_enc.fit_transform(metro_data[['time']])
# print(metro_data['time'].value_counts())
# metro_data[]
metro_data['day'] = pd.to_datetime(metro_data['date_time']).dt.day
metro_data['month'] = pd.to_datetime(metro_data['date_time']).dt.month
metro_data['year'] = pd.to_datetime(metro_data['date_time']).dt.year
metro_data.drop('date_time', axis=1)

# print(metro_data)
featurenames = ['traffic_volume',
                        'holiday',
                        'temp', #this one
                        'rain_1h', #this one
                        'snow_1h', #this one
                        'clouds_all',
                        'weather_main',
                        'weather_description',
                        'day',
                        'month',
                        'year',
                        'time']
metro_data = metro_data[featurenames]
# print(metro_data)
npdata = metro_data.to_numpy()
rng1 = np.random.default_rng(12345)
npdata = rng1.choice(npdata, 10000)
print(npdata)

[[8.3200e+02 7.0000e+00 2.8975e+02 ... 5.0000e+00 2.0170e+03 4.0000e+00]
 [2.3970e+03 7.0000e+00 2.6756e+02 ... 1.2000e+01 2.0130e+03 1.2000e+01]
 [4.2660e+03 7.0000e+00 2.8150e+02 ... 1.0000e+01 2.0170e+03 1.0000e+01]
 ...
 [5.7280e+03 7.0000e+00 2.7508e+02 ... 1.0000e+00 2.0170e+03 8.0000e+00]
 [1.4420e+03 7.0000e+00 2.7753e+02 ... 4.0000e+00 2.0160e+03 6.0000e+00]
 [2.9100e+02 7.0000e+00 2.5277e+02 ... 2.0000e+00 2.0130e+03 2.0000e+00]]


In [6]:
random.seed(50)
test_indices = np.unique(np.array([random.randint(0,npdata.shape[0]-1) for k in range(0,math.ceil(0.2*npdata.shape[0]))]))
# print(test_indices)
test_data = npdata[test_indices, :]
test_X = test_data[:,1:]
test_transformer = preprocessing.MinMaxScaler().fit(test_X)
normalized_test_X = test_transformer.transform(test_X)
real_mpg = test_data[:, 0]

mu = 0
features = [[1]]
# features = [[1], [2], [3], [1,2], [1,3], [2,3], [1,2,3]] 
input_variances = np.arange(0.00,0.02,0.01)
iterations = 1

results = np.zeros([iterations, test_X.shape[0]])
output_variances = np.zeros(input_variances.size)
plot_matrix = np.zeros([len(features), input_variances.size])

for j in range(0, len(features)):
    k = 0
    for sig in input_variances:
        rng = np.random.default_rng(12345)
        for i in range(0, iterations):
            data = copy.deepcopy(npdata)
            training_data = np.delete(data, test_indices, axis=0)
            rows = training_data.shape[0]
            noise_indices = rng.choice(rows-1, math.ceil(0.1*rows), replace=False)
            training_X = training_data[:, 1:]
            training_transformer = preprocessing.StandardScaler().fit(training_X)
            normalized_training_X = training_transformer.transform(training_X)
            for el in features[j]:
                noises = rng.normal(mu, sig, noise_indices.shape)
                normalized_training_X[noise_indices, el] += noises
            training_y = training_data[:, 0]
            ######################################################### LINEAR REGRESSION
            # clf = linear_model.LinearRegression()
            ######################################################### DECISION TREE
            clf = tree.DecisionTreeRegressor(random_state=42)
            ######################################################### NEURAL NET
            # clf = neural_network.MLPRegressor(random_state=42, max_iter=500)
            #########################################################
            clf = clf.fit(normalized_training_X, training_y)
            dot_data = tree.export_graphviz(clf, out_file='tree.dot')
            (graph,) = pydot.graph_from_dot_file('tree.dot')
            graph.write_png('treeeeeee.png')
            prediction_mpg = clf.predict(normalized_test_X)
            results[i] = prediction_mpg
        variances = np.array([np.var(results[:, k]) for k in range(0, results.shape[1])])
        mean_output_variance = np.mean(variances)
        print(mean_output_variance)
        output_variances[k] = mean_output_variance
        plot_matrix[j] = output_variances
        k += 1
# print(output_variances)
print("done")
now = datetime.now()
timestamp = datetime.timestamp(now)
experiment_name = "lin reg extra"
path = 'results/metro/'
np.savetxt(path + experiment_name + str(timestamp) + '.csv', plot_matrix, delimiter=",")

FileNotFoundError: [WinError 2] "dot" not found in path.

In [ ]:
#Plot the outcomes
t=0
plot_matrix[:,1] = plot_matrix[:,2]
for arr in plot_matrix:
    plt.plot(input_variances, arr, label="line " + str(features[t]))
    t+=1
plt.xlabel('input variance')
plt.ylabel('output variance')
plt.legend()
plt.savefig(path + experiment_name + str(timestamp) + '.png')
plt.show()


In [ ]:
# random.seed(50)
# test_indices = np.unique(np.array([random.randint(0,npdata.shape[0]-1) for k in range(0,1000)]))
# # print(test_indices)
# test_data = npdata[test_indices, :]
# test_X = test_data[:,1:]
# real_mpg = test_data[:, 0]
# training_data = np.delete(npdata, test_indices, axis=0)
# training_X = training_data[:, 1:]
# training_y = training_data[:, 0]
# # clf = linear_model.LinearRegression()
# # clf = tree.DecisionTreeRegressor(random_state=42)
# clf = neural_network.MLPRegressor(random_state=42, max_iter=500)
# clf = clf.fit(training_X, training_y)
# prediction_mpg = clf.predict(test_X)
# print(prediction_mpg-real_mpg)

